In [ ]:
# In linux 
# cd ~
# wget https://raw.githubusercontent.com/midwire/free_zipcode_data/develop/all_us_zipcodes.csv
# hdfs dfs -copyFromLocal ./all_us_zipcodes.csv /input


In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
# local[*] - as many cpu available
sc = SparkContext("local[*]", "ZipCodesPartitions")


In [10]:
# fileRdd = sc.textFile("hdfs://192.168.93.128:9000/input/all_us_zipcodes.csv")
fileRdd = sc.textFile("hdfs://192.168.93.128:9000/input/all_us_zipcodes.csv")

header = fileRdd.first()
print(header)
fileRdd = fileRdd.filter(lambda line: line != header)\
                 .map(lambda line: line.strip().split(","))\
                 .map(lambda data: tuple(data))

# we need upto 4 partitions, compute portable_hash [default] based on first member of the tuple
fileRdd.partitionBy(4)    
# fileRdd.take(4)

def showPartData(partitionData):
    if (partitionData is None):
        return
    
    i = 0 
    
    print ("Part ==========")
    try:
        for element in partitionData:
            print (element)
            i += 1
            if (i == 10):
                break
    except:
        print("ERROR")
        
    return partitionData

print("NumPartitions", fileRdd.getNumPartitions())
fileRdd = fileRdd.mapPartitions(showPartData)

k = fileRdd.collect() # reads from all parititions
fileRdd.take(4)

code,city,state,county,area_code,lat,lon
NumPartitions 2


[('01010', 'Brimfield', 'MA', 'HAMPDEN', '', '42.108585', '-72.20448'),
 ('01011', 'Chester', 'MA', 'HAMPDEN', '', '42.294259', '-72.952776'),
 ('01012', 'Chesterfield', 'MA', 'HAMPSHIRE', '', '42.392274', '-72.825607'),
 ('01013', 'Chicopee', 'MA', 'HAMPDEN', '', '42.161492', '-72.667341')]

In [ ]:
from pyspark.rdd import portable_hash

print ("Hash", portable_hash('MA'))

In [ ]:

def showPartData(partitionData):
    if (partitionData is None):
        return
    
    i = 0 
    
    print ("Part ==========")
    try:
        for element in partitionData:
            print (element)
            i += 1
            if (i == 10):
                break
    except:
        print("ERROR")
        
    return partitionData

In [18]:
# fileRdd = sc.textFile("hdfs://192.168.93.128:9000/input/all_us_zipcodes.csv")
fileRdd = sc.textFile("hdfs://192.168.93.128:9000/input/all_us_zipcodes.csv")

header = fileRdd.first()
print(header)
# keyed Rdd, where key is the state code, data[2] is a state code NY, CO
fileRdd = fileRdd.filter(lambda line: line != header)\
                 .map(lambda line: line.strip().split(","))\
                 .map(lambda data: (data[2], tuple(data)))

def byStateCode(state):
    if (state == "NY"):
        return 1
    #print("**", state)
    return hash(state)

# we need upto 4 partitions
# by default paritionBy uses portable_hash, now we are overwriting that by passing custom partioner

fileRdd = fileRdd.partitionBy(52, byStateCode)    
# fileRdd.take(4)


print("NumPartitions", fileRdd.getNumPartitions())
fileRdd = fileRdd.mapPartitions(showPartData)

k = fileRdd.collect() # reads from all parititions
#fileRdd.take(4)

code,city,state,county,area_code,lat,lon
NumPartitions 52
